In [16]:
import tabula
import pandas as pd

In [17]:
dfs = tabula.read_pdf('./Events.pdf', pages="all")
len(dfs)

2

In [18]:
df = dfs[0]
df.columns = ["Name", "Onsite Only", "Level", "Type", "# of Members", "State Entry Limit", "National Entry Limit"]
df = df[1:].reset_index(drop=True)

In [19]:
df1 = dfs[1]
df1

,Technology Problem Solving,High School,bring toolbox,Team,2,2 per Chapter,1 per Chapter
0,Transportation Modeling,High School,NaN,Individual,1,3 per Chapter,1 per Chapter
1,Trebuchet,High School,NaN,Team,2-4,3 per Chapter,Not offered - State only
2,VEX Robotics,High School,NaN,Team,2-6,3 per Chapter,Not offered - State only
3,Video Game Design,High School,NaN,Team,2-6,3 per Chapter,5 per State
4,Virtual Reality Visualization (VR),High School,NaN,Ind or Team,1-6,3 per Chapter,1 per Chapter
5,Webmaster,High School,NaN,Team,2-6,2 per Chapter,1 per Chapter


In [20]:
first_entry = df1.columns
df1.loc[len(df)] = first_entry
df1.columns = ["Name", "Onsite Only", "Level", "Type", "# of Members", "State Entry Limit", "National Entry Limit"]

In [21]:
df = pd.concat([df, df1])
df.reset_index(inplace=True)
df.drop(["index", "Onsite Only"], axis=1, inplace=True)

In [22]:
df.head()

,Name,Level,Type,# of Members,State Entry Limit,National Entry Limit
0,Animation,High School,Ind or Team,1-6,3 per Chapter,Not offered - State only
1,Animatronics,High School,Team,2-3,3 per Chapter,3 per State
2,Architectural Design,High School,Ind or Team,1-6,3 per Chapter,1 per Chapter
3,Audio Podcasting,High School,Ind or Team,1-6,3 per Chapter,3 per State
4,Biotechnology Design,High School,Team,2-6,3 per Chapter,1 per Chapter


In [23]:
def parseMembers(row):
    ret = {}
    try:
        members = row["# of Members"]
        ret["minMembers"] = int(members[0])
        if("-" in members):
            ret["maxMembers"] = int(members[2])
        else:
            ret["maxMembers"] = int(members[0])

        return pd.Series(ret)
    except:
        return pd.Series({"minMembers": -1, "maxMembers": -1})

df[["minMembers", "maxMembers"]] = df.apply(parseMembers, axis=1)

In [24]:
def teamsAtStates(row):
    ret= {}
    state = row["State Entry Limit"]

    if(state == "No Limit"):
        ret["maxTeams"] = 10000
    else:
        ret["maxTeams"] = int(state[0])

    return pd.Series(ret)

df["maxTeams"] = df.apply(teamsAtStates, axis=1)

In [25]:
df.head()

,Name,Level,Type,# of Members,State Entry Limit,National Entry Limit,minMembers,maxMembers,maxTeams
0,Animation,High School,Ind or Team,1-6,3 per Chapter,Not offered - State only,1,6,3
1,Animatronics,High School,Team,2-3,3 per Chapter,3 per State,2,3,3
2,Architectural Design,High School,Ind or Team,1-6,3 per Chapter,1 per Chapter,1,6,3
3,Audio Podcasting,High School,Ind or Team,1-6,3 per Chapter,3 per State,1,6,3
4,Biotechnology Design,High School,Team,2-6,3 per Chapter,1 per Chapter,2,6,3


In [26]:
import firebase_admin
from firebase_admin import firestore
import dotenv
import os

In [27]:
dotenv.load_dotenv(".env.local")

True

In [28]:
cred = firebase_admin.credentials.Certificate('./service_account.json')

try:
    firebase = firebase_admin.initialize_app(cred)
except:
    pass

db = firestore.client()

In [29]:
def add_to_firebase(row):
    data = {"name": row["Name"], "minMembers": row["minMembers"], "maxMembers": row["maxMembers"], "maxTeams": row["maxTeams"], "teams": []}
    ret = {}
    try:
        db.collection("Events").add(data)
        ret["Added"] = True
    except:
        ret["Added"] = False

    return pd.Series(ret)

df["Added"] = df.apply(add_to_firebase, axis=1)

In [30]:
df

,Name,Level,Type,# of Members,State Entry Limit,National Entry Limit,minMembers,maxMembers,maxTeams,Added
0,Animation,High School,Ind or Team,1-6,3 per Chapter,Not offered - State only,1,6,3,True
1,Animatronics,High School,Team,2-3,3 per Chapter,3 per State,2,3,3,True
2,Architectural Design,High School,Ind or Team,1-6,3 per Chapter,1 per Chapter,1,6,3,True
3,Audio Podcasting,High School,Ind or Team,1-6,3 per Chapter,3 per State,1,6,3,True
4,Biotechnology Design,High School,Team,2-6,3 per Chapter,1 per Chapter,2,6,3,True
5,Board Game Design,High School,Team,2-6,3 per Chapter,1 per Chapter,2,6,3,True
6,Career Prep,High School,Individual,1,3 per Chapter,Not offered - State only,1,1,3,True
7,Children's Stories,High School,Ind or Team,1-6,3 per Chapter,3 per State,1,6,3,True
8,Coding,High School,X Team,2,3 per Chapter,3 per State,2,2,3,True
9,"Computer-Aided Design 2D, Architecture",High School,X Individual,1,3 per Chapter,2 per State,1,1,3,True
